In [ ]:
import cv2
import numpy as np
import os
import gc
from scipy import ndimage

In [ ]:
def count_holes(mask):
    # Find the main contour (largest white area)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return 0  # No white area found

    main_contour = max(contours, key=cv2.contourArea)

    # Create a mask only for the main white area
    main_mask = np.zeros_like(mask)
    cv2.drawContours(main_mask, [main_contour], -1, 255, thickness=cv2.FILLED)

    # Invert the original mask
    inverted = cv2.bitwise_not(mask)

    # Get only the holes inside the main area
    holes = cv2.bitwise_and(inverted, main_mask)

    # Find contours of the holes
    hole_contours, _ = cv2.findContours(holes, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    return len(hole_contours)

In [ ]:
# Folder paths
png_folder_path = r'path\to\segmentation_masks_folder_sagittal_plane'

cranium_mask_folder = os.path.join(png_folder_path, 'cranium_mask')
bone_mask_folder = os.path.join(png_folder_path, 'bone_mask')
diploe_mask_folder = os.path.join(png_folder_path, 'diploe_mask')
# colormap_folder = os.path.join(png_folder_path, 'Colormap')
# bone_seg_folder = os.path.join(png_folder_path, 'Bone Segmentation')
# diploe_seg_folder = os.path.join(png_folder_path, 'Diploe Segmentation')

# Create folders if they do not exist
os.makedirs(png_folder_path, exist_ok=True)
os.makedirs(cranium_mask_folder, exist_ok=True)
os.makedirs(bone_mask_folder, exist_ok=True)
os.makedirs(diploe_mask_folder, exist_ok=True)
# os.makedirs(colormap_folder, exist_ok=True)
# os.makedirs(bone_seg_folder, exist_ok=True)
# os.makedirs(diploe_seg_folder, exist_ok=True)

# Configuration parameters: 
### == CHOOSE THE SLICE RANGE == ###
start_slice = 126
end_slice = 383
### ============================ ###

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 50, 0.5)
k = 2

for slice_index in range(start_slice, end_slice + 1):

    print(f"In progress: Slice_{slice_index}", end='\r')

    image_path = f'path\to\png_folder_sagittal_plane\Slice_{slice_index}.png'
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    #### STEP 1: SEGMENTATION USING K-MEANS CLUSTERING ####

    # Mask to exclude black pixels > 20 (background)
    non_zero_mask = image > 20

    # Apply K-means only on non-zero pixels (k = 2)
    pixel_values = image[non_zero_mask].reshape((-1, 1)).astype(np.float32)
    _, labels, centers = cv2.kmeans(pixel_values, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    centers = np.uint8(centers)

    # Sort clusters by intensity
    sorted_indices = np.argsort(centers.flatten())

    # Map clusters to 127 and 255
    mapped_values = np.zeros_like(labels.flatten(), dtype=np.uint8)
    mapped_values[labels.flatten() == sorted_indices[0]] = 127
    mapped_values[labels.flatten() == sorted_indices[1]] = 255

    # Create final image: 0 (background), 127 (diploe), 255 (bone)
    segmented_image = np.zeros_like(image, dtype=np.uint8)
    segmented_image[non_zero_mask] = mapped_values

    #### STEP 2: CREATE FULL CRANIUM MASK ####

    # Create binary mask: all values > 0 become 255
    binary_mask = np.where(segmented_image > 0, 255, 0).astype(np.uint8)

    kernel_size = 1
    max_kernel_size = 60  # Set a reasonable max limit to avoid infinite loops

    while kernel_size <= max_kernel_size:
        kernel = np.ones((kernel_size, kernel_size), np.uint8)
        closed_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel)

        num_holes = count_holes(closed_mask)

        if num_holes == 0:
            break

        kernel_size += 2

    #### STEP 4: CREATE BONE MASK ####

    # Find unique cluster values
    unique_values = np.unique(segmented_image)

    # Identify the two highest values
    bone_value = unique_values[-1]

    # Create binary mask
    bone_mask = np.isin(segmented_image, bone_value).astype(np.uint8)*255

    #### STEP 5: CREATE DIPLOE MASK ####

    diploe_mask = cv2.subtract(closed_mask, bone_mask)

    # 1. Find contour
    contours, _ = cv2.findContours(diploe_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 2. Create blank image
    contour_full = np.zeros_like(diploe_mask)

    # 3. Draw thick contour (half inside, half outside)
    cv2.drawContours(contour_full, contours, -1, color=255, thickness=1)

    # 4. Keep only the inner part of the contour
    external_contour = cv2.bitwise_and(contour_full, diploe_mask)

    # 5a. Draw diploe mask contour
    contours, _ = cv2.findContours(diploe_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 5b. Create empty mask to draw contours
    border_mask = np.zeros_like(diploe_mask)

    # 6. Draw thick contours on border_mask
    cv2.drawContours(border_mask, contours, -1, color=255, thickness=15)

    # 7. Subtract contour from diploe_mask
    inner_mask = cv2.subtract(diploe_mask, border_mask)

    # 8. Create image of certain areas
    sure_fg = cv2.bitwise_or(external_contour, inner_mask)

    # 9. Create image of uncertain areas
    unknown = cv2.subtract(diploe_mask, sure_fg)

    # 10. Label the two components
    labels = np.zeros_like(diploe_mask)
    labels[external_contour == 255] = 1
    labels[inner_mask == 255] = 2

    # 11. Apply Watershed algorithm
    markers = labels + 1
    markers[unknown == 255] = 0

    img_color = cv2.cvtColor(diploe_mask, cv2.COLOR_GRAY2BGR)
    markers = cv2.watershed(img_color, markers.astype(np.int32))

    # 12. Include both region 2 (markers == 2) and borders (markers == -1)
    noise_contour_mask = np.logical_or(markers == 2, markers == -1).astype(np.uint8) * 255

    # 13. Create diploe mask without border artifacts
    diploe_minus_region2 = cv2.subtract(diploe_mask, noise_contour_mask)
    filled_diploe_mask = ndimage.binary_fill_holes(diploe_minus_region2).astype(np.uint8)*255

    # Apply median filtering
    final_diploe_mask = cv2.medianBlur(filled_diploe_mask, 3)  # 3x3 kernel

    #### STEP 6: FINAL BONE MASK ####

    final_bone_mask = cv2.subtract(closed_mask, final_diploe_mask)

    #### STEP 8: SAVE OUTPUT IMAGES ####

    cranium_mask_path = os.path.join(cranium_mask_folder, f'Slice_{slice_index}_cranium_mask.png')
    bone_mask_path = os.path.join(bone_mask_folder, f'Slice_{slice_index}_bone_mask.png')
    diploe_mask_path = os.path.join(diploe_mask_folder, f'Slice_{slice_index}_diploe_mask.png')
    # bone_seg_path = os.path.join(bone_seg_folder, f'Slice_{slice_index}_bone_seg.png')
    # diploe_seg_path = os.path.join(diploe_seg_folder, f'Slice_{slice_index}_diploe_seg.png')
    
    cv2.imwrite(cranium_mask_path,  closed_mask)
    cv2.imwrite(bone_mask_path, final_bone_mask)
    cv2.imwrite(diploe_mask_path, final_diploe_mask)
    # cv2.imwrite(bone_seg_path, bone_seg)
    # cv2.imwrite(diploe_seg_path, diploe_seg)

    # Reset all local variables at the end of the slice
    cv2.setRNGSeed(1234)  # Reset RNG (important!)
    del image, segmented_image, binary_mask, pixel_values
    del labels, centers, mapped_values, closed_mask
    del bone_mask, diploe_mask, contour_full, external_contour
    del border_mask, inner_mask, sure_fg, unknown
    del markers, img_color, noise_contour_mask
    del diploe_minus_region2, final_diploe_mask, final_bone_mask
    gc.collect()

    #### STEP 9: SAVE COLORMAP IMAGES ####

    # image_bgr = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

    # # Create empty overlay
    # overlay = image_bgr.copy()

    # # BGR colors
    # red = (0, 0, 255)
    # green = (0, 255, 0)

    # # Apply color to masks
    # overlay[bone_mask > 0] = red
    # overlay[final_diploe_mask > 0] = green

    # # Merge original image and overlay with transparency
    # alpha = 0.3
    # final_image_bgr = cv2.addWeighted(image_bgr, 1.0, overlay, alpha, 0)

    # # Save with cv2
    # colormap_path = os.path.join(colormap_folder, f'Slice_{slice_index}_colormap.png')
    # cv2.imwrite(colormap_path, final_image_bgr)